# Creating a recommender system with RNNs

We will observe the history of items that customers have bought from our online store. Our objective is to predict the next item that a customer will buy, given their purchase history.

**Question 1**: Assuming you can predict well what customers are going to buy when visiting our store. What can you do with this information in order to improve the profits of our online store?

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras.layers import Embedding, Conv1D, BatchNormalization, GRU, Dense
from sklearn.model_selection import train_test_split

/Users/pengjunming/Desktop/Anaconda/anaconda3/envs/myenv/lib/python3.10/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/Users/pengjunming/Desktop/Anaconda/anaconda3/envs/myenv/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 1. The data

Each entry in the dataset corresponds to a combination of customer and item bought. Per customer, items are arranged based on the visits to our online store (e.g., customer 17850 first bought item 0, then item 399, then item 505, etc.)

In [2]:
df = pd.read_csv('https://www.dropbox.com/s/4kicl5okwlmst5i/online_retail.csv?dl=1')
df.head()

,CustomerID,StockCode
0,17850.0,0
1,17850.0,399
2,17850.0,505
3,17850.0,227
4,17850.0,253


We will later use zero-padding to get sequences of equal length. Hence, we should avoid items with name "0" and instead shift all items by 1:

In [3]:
df['StockCode'] = df['StockCode'] + 1

In [4]:
number_items = len(df['StockCode'].unique())
number_items

998

We convert the dataframe into a list of sequences, where each customer corresponds to one sequence of items bought.

In [5]:
sequences = []
for customer in df['CustomerID'].unique():
    temp = df[df['CustomerID'] == customer]
    sequences.append(temp['StockCode'].tolist())

Some sequences are much longer than others, so we will only consider sequences of a certain length. In particular, we pick here approximately the 90% quantile to cut off sequences of purchases.

In [6]:
np.quantile([len(seq) for seq in sequences],0.9)

158.0

In [7]:
max_length = 160
sequences = [seq[:min(max_length,len(seq))] for seq in sequences]

We also add "padding" to make sequences of equal length (to train our model, each sequence within a mini-batch has to have the same length. Since we don't want to have a lot of work splitting the data into mini-batches, we will just equalize everything). Note that we will need to tell our algorithm later to ignore padded values when it comes to loss-computation.

In [8]:
sequences[10]

[102, 124, 367, 12, 16, 46, 500, 177, 188, 231]

In [9]:
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding="pre")

In [10]:
sequences[10]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0, 102, 124, 367,  12,  16,  46,
       500, 177, 188, 231], dtype=int32)

Next, we split the data into training, validation, and testing (randomly):

In [ ]:
X_train, X_other = train_test_split(sequences,train_size=0.8)
X_valid, X_test = train_test_split(X_other,train_size=0.5)
print(X_train.shape)

Finally, we build our target (y) based on a sequence-to-sequence approach. That is, for each sequence of inputs, we predict a sequence of outputs.

**Question 2**: Construct the variables `y_train`, `y-valid`, and `y_test`. Then, modify `X_train`, `X_valid`, and `X_test` appropriately to ensure that input and output sequences are of the same length. *Hint*: After any item has been bought, we want to predict the next item bought by the same customer.

Check your sequence lengths:

In [ ]:
print(X_train.shape)
print(y_train.shape)

## 2. Building a model

We now build a model that takes as input a sequence of orders by one customer and outputs the predictions for the next time step. Instead of directly using our sequences as inputs to a recurrent layer, we will use an `Embedding` layer.

**Question 3**: In your own words, describe what (word) embeddings do, and why we use them in deep learning. A good resource is the accompanying book "Deep Learning with Python" (2nd edition) by Francois Chollet, available online through the City University Library. You might want to check the part "Understanding Word Embeddings" within Chapter 11.3.3.

In [ ]:
model = tf.keras.Sequential([
    Embedding(input_dim=number_items+1, output_dim=6, input_shape=[None], mask_zero=True),
    Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
    BatchNormalization(),
    GRU(64, return_sequences=True, dropout = 0.2),
    BatchNormalization(),
    Dense(number_items+1, activation="softmax")
])
model.summary()

We want to add our own metric, to capture how well we're doing on the last prediction (that's the only one that matters after all). In particular, we will see whether the product the customer bought is within the 5 products we gave the highest probability in our prediction.

**Question 4**: Define a function `last_time_step_top_5` that takes the inputs `y_true` and `y_pred` and computes the `tf.keras.metrics.sparse_top_k_categorical_accuracy` between `y_true` and `y_pred` *for the last entry of each sequence*. Note that `sparse_top_k_categorical_accuracy` takes as input the (modified) `y_true` and `y_pred`, as well as a value `k`.

We are now ready to train the model:

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
                metrics = [last_time_step_top_5])
log = model.fit(X_train, y_train, epochs=20,
                validation_data = (X_valid,y_valid))

plt.plot(log.history['last_time_step_top_5'],label = "actual in top 5 - training",color='green')
plt.plot(log.history['val_last_time_step_top_5'], label = "actual in top 5 - validation",color='grey')
plt.legend()
ax = plt.gca()
plt.show()

On the one hand, this doesn't sound too impressive. On the other hand, keep in mind that we have looked at raw items, and 1000 of them (while only having the buying history of 3500 customers).

**Question 5**: Can you do better? Go through the frameworks we have discussed in class in order to generate an improved model. A few hints:
- Before thinking about our framework for improving bias and variance, note that the model does not yet really overfit
- While we generally don't stack recurrent layers too deeply for computational reasons, we are currently only using a single one
- Consider the specific type of dropout regularization relevant for RNNs
- Aside from the typical suspects for parameters to modify, the number of dimensions of the embedding usually has a big influence

At the end of your improvement process, evaluate your model on the test set.